In [1]:
import pandas as pd
import numpy as npplò7

In [2]:
df = pd.read_csv('train.csv')

In [3]:
print(df.shape)
df.head(1)

(1804874, 45)


,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4


In [4]:
df_test = pd.read_csv('test.csv')
print(df_test.shape)
df_test.head(2)

(97320, 2)


,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...


In [5]:
sample = pd.read_csv('sample_submission.csv')
print(sample.shape)
sample.sample(2)

(97320, 2)


,id,prediction
9203,7009203,0.0
56035,7056035,0.0


In [6]:
# Deleting all the columns with null values
columns_to_drop = df.columns[df.isnull().sum()>0]

In [7]:
df = df.drop(columns_to_drop,axis=1)
print(df.isnull().sum()[:2])

id        0
target    0
dtype: int64


In [8]:
df.head(2)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,created_date,publication_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:41.987077+00,2,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.0,Thank you!! This would make my life a lot less...,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:42.870083+00,2,2006,rejected,0,0,0,0,0,0.0,0,4


In [9]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/luigibungaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
df['comment_text'][0]

"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"

In [ ]:
# x = word_tokenize(df['comment_text'][0])

In [11]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
x = tokenizer.tokenize(df['comment_text'][0])

In [12]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luigibungaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
filtered_sent=[]
for w in x:
    if w not in stop_words:
        filtered_sent.append(w)

In [14]:
embedding = pd.read_csv('numberbatch-en-17.06.txt',delimiter=' ',header=None)
print(embedding.shape)
embedding.sample(2)

(417194, 301)


,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
15586,anguilla,-0.1375,-0.0025,0.0629,0.0178,-0.0375,0.0451,0.0994,0.1234,-0.0238,...,-0.0029,0.0484,-0.0511,0.0313,0.0055,0.0216,0.0552,0.0116,0.1056,-0.0649
132071,felinity,-0.0217,-0.0653,0.0568,0.1026,0.1747,0.0759,-0.0519,0.0207,-0.1229,...,-0.0006,0.0326,-0.0024,0.0755,-0.0034,-0.0273,0.0556,-0.0104,0.0788,-0.0143


In [15]:
def preprocessing_string(string):
    x = tokenizer.tokenize(string)
    filtered_sent=[]
    for w in x:
        if w not in stop_words:
            filtered_sent.append(w)
    return filtered_sent

In [16]:
fgh = preprocessing_string("This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!")

In [17]:
len(df['comment_text'])

1804874

In [18]:
# text value
x = df['comment_text'][0]
x

"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"

In [19]:
# target value
df[df['comment_text']==x]['target'].values[0]

0.0

In [53]:
data = []
labels = []
j=0
for i in df['comment_text'][:20000]:
    j+=1
    print(j, end='\r')
    preprocessed_string = preprocessing_string(i)
    # print(preprocessed_string)
    if embedding[0].isin(list(preprocessed_string)).sum()==0:
        data.append(np.array([0.1]*300))
    else:
        embedded_vector = embedding[embedding[0].isin(preprocessed_string)][embedding.columns[1:]].mean().values
        # print(type(embedded_vector))
        data.append(embedded_vector)
data = np.array(data)
print(data.shape)
labels = df.target
# labels

(20000, 300)


In [54]:
df_final = pd.DataFrame(data,columns=None)
df_final['target']=labels

In [55]:
df_final.to_pickle('df_final.pkl')

In [56]:
df_final = pd.read_pickle('df_final.pkl')

In [57]:
print(df.shape)

(1804874, 20)


In [58]:
df_final.head(2)

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,target
0,0.159480,0.136200,0.025170,-0.009980,0.021080,-0.01213,0.121980,-0.069570,-0.049690,0.092570,...,-0.027710,-0.002890,0.007430,0.038270,0.034800,-0.005830,-0.010490,-0.011680,-0.023620,0.0
1,0.186845,0.125036,0.017573,-0.036018,0.021982,-0.09090,0.096555,0.006936,-0.074545,0.067818,...,-0.021809,-0.013818,-0.020982,0.009927,0.013982,-0.019973,0.006991,0.022091,0.007345,0.0


In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_final[df_final.columns[:-1]], df_final['target'],
                                                    test_size = 0.2, random_state = 0)

In [60]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [61]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 400, init = 'uniform', activation = 'relu', input_dim = 300))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 200, init = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 100, init = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 50, init = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

adam = keras.optimizers.Adam(lr=0.00011)
# Compiling the ANN
classifier.compile(optimizer = adam, loss = 'mean_squared_error', metrics = ['mae'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 100, epochs = 200)

/Users/luigibungaro/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=300, units=400, kernel_initializer="uniform")`
  """
/Users/luigibungaro/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200, kernel_initializer="uniform")`
  
/Users/luigibungaro/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="uniform")`
  # This is added back by InteractiveShellApp.init_path()
/Users/luigibungaro/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=50, kernel_initializer="uniform")`
  
/Users/luigibungaro/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: U

Epoch 1/200
16000/16000 [==============================] - 2s 104us/step - loss: 0.1398 - mean_absolute_error: 0.3330
Epoch 2/200
16000/16000 [==============================] - 1s 66us/step - loss: 0.0318 - mean_absolute_error: 0.1231: 0s - loss: 0.0318 - mean_absolute_error: 0.123
Epoch 3/200
16000/16000 [==============================] - 1s 67us/step - loss: 0.0296 - mean_absolute_error: 0.1212
Epoch 4/200
16000/16000 [==============================] - 1s 66us/step - loss: 0.0272 - mean_absolute_error: 0.1166
Epoch 5/200
16000/16000 [==============================] - 1s 67us/step - loss: 0.0249 - mean_absolute_error: 0.1104
Epoch 6/200
16000/16000 [==============================] - 1s 67us/step - loss: 0.0229 - mean_absolute_error: 0.1036
Epoch 7/200
16000/16000 [==============================] - 1s 71us/step - loss: 0.0213 - mean_absolute_error: 0.0971
Epoch 8/200
16000/16000 [==============================] - 1s 81us/step - loss: 0.0201 - mean_absolute_error: 0.0920
Epoch 9/200
160

16000/16000 [==============================] - 1s 86us/step - loss: 0.0039 - mean_absolute_error: 0.0320
Epoch 70/200
16000/16000 [==============================] - 1s 69us/step - loss: 0.0039 - mean_absolute_error: 0.0315
Epoch 71/200
16000/16000 [==============================] - 1s 71us/step - loss: 0.0038 - mean_absolute_error: 0.0312
Epoch 72/200
16000/16000 [==============================] - 1s 71us/step - loss: 0.0037 - mean_absolute_error: 0.0308
Epoch 73/200
16000/16000 [==============================] - 1s 82us/step - loss: 0.0037 - mean_absolute_error: 0.0307
Epoch 74/200
16000/16000 [==============================] - 1s 71us/step - loss: 0.0036 - mean_absolute_error: 0.0301
Epoch 75/200
16000/16000 [==============================] - 1s 78us/step - loss: 0.0037 - mean_absolute_error: 0.0308
Epoch 76/200
16000/16000 [==============================] - 1s 83us/step - loss: 0.0036 - mean_absolute_error: 0.0297
Epoch 77/200
16000/16000 [==============================] - 1s 88us/s

16000/16000 [==============================] - 1s 71us/step - loss: 0.0021 - mean_absolute_error: 0.0191
Epoch 139/200
16000/16000 [==============================] - 1s 69us/step - loss: 0.0022 - mean_absolute_error: 0.0195
Epoch 140/200
16000/16000 [==============================] - 1s 70us/step - loss: 0.0021 - mean_absolute_error: 0.0189
Epoch 141/200
16000/16000 [==============================] - 1s 89us/step - loss: 0.0020 - mean_absolute_error: 0.0182
Epoch 142/200
16000/16000 [==============================] - 1s 88us/step - loss: 0.0021 - mean_absolute_error: 0.0193
Epoch 143/200
16000/16000 [==============================] - 1s 87us/step - loss: 0.0020 - mean_absolute_error: 0.0181
Epoch 144/200
16000/16000 [==============================] - 2s 98us/step - loss: 0.0020 - mean_absolute_error: 0.0180
Epoch 145/200
16000/16000 [==============================] - 1s 72us/step - loss: 0.0020 - mean_absolute_error: 0.0182
Epoch 146/200
16000/16000 [==============================] - 1

In [62]:
y_pred = classifier.predict(X_test)

In [63]:
test = pd.read_csv('k_toxicity/test.csv')

In [64]:
print(test.shape)
test.head(2)

(97320, 2)


,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...


In [65]:
data = []
labels = []
j=0
for i in test['comment_text']:
    j+=1
    print(j, end='\r')
    preprocessed_string = preprocessing_string(i)
    # print(preprocessed_string)
    if embedding[0].isin(list(preprocessed_string)).sum()==0:
        data.append(np.array([0.1]*300))
    else:
        embedded_vector = embedding[embedding[0].isin(preprocessed_string)][embedding.columns[1:]].mean().values
        # print(type(embedded_vector))
        data.append(embedded_vector)
data = np.array(data)
print(data.shape)
labels = df.target

(97320, 300)


In [66]:
test_final_df = pd.DataFrame(data,columns=None)
test_final_df.shape

(97320, 300)

In [67]:
test_final_df.to_pickle('df_final_test.pkl')

In [47]:
test_final_df = pd.read_pickle('df_final_test.pkl')

In [68]:
print(test_final_df.shape)
test_final_df.head(2)

(97320, 300)


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.135491,0.096145,-0.013600,-0.095782,-0.018945,0.033373,0.074082,-0.002736,-0.035864,0.025673,...,0.028473,-0.003955,0.012527,0.017855,-0.007791,0.002627,0.013336,0.004036,-0.0041,0.001373
1,0.077924,0.053862,-0.011543,-0.057910,-0.086705,0.023605,0.095248,0.037181,-0.036629,-0.000733,...,0.004733,-0.018214,0.020190,0.004210,-0.018438,0.003914,0.002695,-0.005829,-0.0140,0.011967


In [69]:
final_pred = classifier.predict(test_final_df)

In [73]:
len(final_pred)
final_pred = [i[0] for i in final_pred]
final_pred[:5]

[0.00083357096, 0.0007523596, 0.18382111, 0.0219495, 0.28862673]

In [71]:
sample_submission = pd.read_csv('k_toxicity/sample_submission.csv')
print(sample_submission.shape)
sample_submission.head(2)

(97320, 2)


,id,prediction
0,7000000,0.0
1,7000001,0.0


In [74]:
sample_submission['prediction'] = final_pred

In [75]:
sample_submission.to_csv('submission_1.csv')